# Launch a Seldon Deployment
> Get an ML endpoint up and running on your cluster!

- toc: true 
- badges: true
- comments: true
- categories: [kubernetes, docker]

### Intro
In this post, we will walkthrough the first steps towards launching your own seldon deployment! 

### Launch cluster
To get started, let's get a cluster up and running. If you followed [part 1]() of this series, you can do so with kind. 
Below, I create a cluster, create a namespace called seldon-intro, then use `kubens` to make seldon-into my default namespace (so I don't have to include it in every command).

In [1]:
!kind create cluster 
!kubectl create namespace seldon-intro
!kubens seldon-intro

Creating cluster "kind" ...
 ✓ Ensuring node image (kindest/node:v1.17.0) 🖼
 ✓ Preparing nodes 📦 7l
 ✓ Writing configuration 📜7l
 ✓ Starting control-plane 🕹️7l
 ✓ Installing CNI 🔌7l
 ✓ Installing StorageClass 💾7l
Set kubectl context to "kind-kind"
You can now use your cluster with:

kubectl cluster-info --context kind-kind

Not sure what to do next? 😅 Check out https://kind.sigs.k8s.io/docs/user/quick-start/
namespace/seldon-intro created
Context "kind-kind" modified.
Active namespace is "seldon-intro".


#### Install seldon-core 
To install seldon-core on the cluster, use helm. To install helm itself, find directions [here](https://helm.sh/), or use `brew install helm` on mac.

Once helm is installed, use it to install seldon-core and seldon-core-operator with the following command:

In [2]:
!helm install seldon-core seldon-core-operator \
    --repo https://storage.googleapis.com/seldon-charts \
    --set usageMetrics.enabled=true \
    --set ambassador.enabled=true \
    --namespace seldon-intro 

NAME: seldon-core
LAST DEPLOYED: Fri Jul 31 07:09:00 2020
NAMESPACE: seldon-intro
STATUS: deployed
REVISION: 1
TEST SUITE: None


To check the install is running correctly, run the following command: 

In [17]:
!kubectl get pods
print("-----")
!kubectl get deployments

NAME                                        READY   STATUS    RESTARTS   AGE
seldon-controller-manager-bc4b8bf64-fssv5   1/1     Running   0          37s
-----
NAME                        READY   UP-TO-DATE   AVAILABLE   AGE
seldon-controller-manager   1/1     1            1           45s


You should see a pod and deployment with `seldon-controller-manager` in the name. This pod and deployment house the seldon-core operator, which is an extensions to to the kubernetes api that allows us to deploy seldon inference graphs later on. We don't need to worry much about these details for now. 

### Build example
I'm taking this example code directly from [seldon-core irisClassifier example](https://github.com/SeldonIO/seldon-core/blob/master/examples/models/sklearn_iris/sklearn_iris.ipynb). 
This is a classic sklearn example we will be able to get up quick. 

In [4]:
!mkdir iris_classifier

mkdir: iris_classifier: File exists


In [5]:
#collapse_show
%%writefile iris_classifier/train_iris.py
import joblib
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn import datasets


OUTPUT_FILE = "iris_classifier/IrisClassifier.sav"


def main():
    clf = LogisticRegression(solver="liblinear", multi_class="ovr")
    p = Pipeline([("clf", clf)])
    print("Training model...")
    p.fit(X, y)
    print("Model trained!")

    print(f"Saving model in {OUTPUT_FILE}")
    joblib.dump(p, OUTPUT_FILE)
    print("Model saved!")


if __name__ == "__main__":
    print("Loading iris data set...")
    iris = datasets.load_iris()
    X, y = iris.data, iris.target
    print("Dataset loaded!")

    main()

Overwriting iris_classifier/train_iris.py


In [6]:
!python iris_classifier/train_iris.py

Loading iris data set...
Dataset loaded!
Training model...
Model trained!
Saving model in iris_classifier/IrisClassifier.sav
Model saved!


In [7]:
#collapse_show
%%writefile iris_classifier/IrisClassifier.py
import joblib

class IrisClassifier(object):

    def __init__(self):
        self.model = joblib.load('IrisClassifier.sav')

    def predict(self,X,features_names):
        return self.model.predict_proba(X)

Overwriting iris_classifier/IrisClassifier.py


I'm going to slightly differ from their example, and use a Dockerfile to create the docker image for this component instead of s2i. Feel free to use s2i directly from their example instead! 

In [8]:
%%writefile iris_classifier/requirements.txt
sklearn
seldon-core

Overwriting iris_classifier/requirements.txt


In [9]:
#collapse_show
%%writefile iris_classifier/Dockerfile
FROM python:3.7-slim
COPY . /app
WORKDIR /app
RUN pip install -r requirements.txt
EXPOSE 5000

# Define environment variable
ENV MODEL_NAME IrisClassifier 
ENV API_TYPE REST
ENV SERVICE_TYPE MODEL 
ENV PERSISTENCE 0

# seldon-core-microservice is a command line tool installed with the seldon-core python libray. You can use this locally as well!
CMD exec seldon-core-microservice $MODEL_NAME $API_TYPE --service-type $SERVICE_TYPE --persistence $PERSISTENCE

Overwriting iris_classifier/Dockerfile


To test this example, let's build and run the docker image! 

In [60]:
#hide_output
!docker build iris_classifier/ -t localhost:5000/iris_ex:latest

Sending build context to Docker daemon  11.26kB
Step 1/10 : FROM python:3.7-slim
 ---> b386e7420fc3
Step 2/10 : COPY . /app
 ---> Using cache
 ---> 350aae576a5c
Step 3/10 : WORKDIR /app
 ---> Using cache
 ---> 752b217417eb
Step 4/10 : RUN pip install -r requirements.txt
 ---> Running in 986fd56d5823
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1315 sha256=16fbb8bcbb99def9b515836945c545b5713029e56ae063a6536c1fecdc81cd2e
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
  Created wheel for jaeger-client: filename=jaeger_client-4.3.0-py3-none-any.whl size=64291 sha256=30e266bd95e0e856bc222089f5d260e2abbe0df294ad95181279d66bcf158641
  Stored in directory: /root/.cache/pip/wheels/4b/b9/d9/efe18893b02a4bc5abb68e0174d4ab10147f7f184dd170758e
  Created wheel for Flask-OpenTracing: filename=Flask_OpenTracing-1.1.0-py3-none-any.whl size=9070 sha256=ec15ccf40ef050f7a3b21bc296dc7521b80d5450653f6f6f9059ea8cd56dc6

In [58]:
!docker run --name "iris_predictor" -d --rm -p 5001:5000 localhost:5000/iris_ex:latest

8cf5c32413068cdec2c01bd6b37d74f7401323ddeebc65c9e33e11b531c0cf0b


You could also remove the -d argument from the above command and run this command in a separate window to see the log output while sending requests to the endpoint. Test the endpoint with the curl below! 

In [19]:
!curl  -s http://localhost:5001/predict -H "Content-Type: application/json" -d '{"data":{"ndarray":[[5.964,4.006,2.081,1.031]]}}'
        

{"data":{"names":["t:0","t:1","t:2"],"ndarray":[[0.9548873249364059,0.04505474761562512,5.792744796895372e-05]]},"meta":{}}


If you see successful output, you have your first seldon-core-microservice up and running! Now, we will deploy this as a simple inference graph on our kubernetes cluster. 
First, let's take down the running docker container:

Next, need to define our deployment configuration file. Here is a seldon config file for our deployment: 

In [59]:
!docker container rm iris_predictor --force

iris_predictor


In [23]:
%%writefile iris_classifier/sklearn_iris_deployment.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: seldon-deployment-example
spec:
  name: sklearn-iris-deployment
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/sklearn-iris:0.1
          imagePullPolicy: IfNotPresent
          name: sklearn-iris-classifier
    graph:
      children: []
      endpoint:
        type: REST
      name: sklearn-iris-classifier
      type: MODEL
    name: sklearn-iris-predictor
    replicas: 1

Writing iris_classifier/sklearn_iris_deployment.yaml


Some important notes about the deployment config: 
* apiVersion: this sends out request to the appropriate endpoint of the kubernets api, which was installed by helm earlier in this tutorial
* kind: tells Kubernetes what kind of resource to create. 
* metadata: add labels, like name, to the deployment
* spec: 
    * predictors: this is a list of predictors to deploy. It is a list because you have the option to create multiple inference graphs in the same spec. This is useful for things like Canary deployment, where you only want a new graph to recieve a small percentage of traffic
        * componentSpecs: add information about the containers that need to be pulled to create our graph. In our case, we only need a single containe to serve our model. If we were creating a more complex inference graph (maybe with a transformer, router, and another model, then we would need to include the docker containers that house them in this section)
        * graph: this is where you define the flow of components. This is easy in our case, there is only one component so we define one endpoint with no children. If there were more compnoents, we would fill out the children componenets in the children attriubte of the head of the graph. Seldon graphs are built implicitly through the use of the children attribute of each node in the graph. 
        
There is one last step to deploy our graph, we must push our docker container to a registry! I am running a local registry with my kind cluster, thanks to the script given [here](https://kind.sigs.k8s.io/docs/user/local-registry/). You can also push to DockerHub as well. 

In [22]:
!docker push localhost:5000/iris_ex:latest

The push refers to repository [localhost:5000/iris_ex]

fd587320: Preparing 
308ed686: Preparing 
63f2d025: Preparing 
f01300cf: Preparing 
a0be9040: Preparing 
1a837902: Preparing 
fd587320: Pushed     276MB/269.6MBPushing  109.8MB/269.6MBPushing    187MB/269.6MBlatest: digest: sha256:696a49e226fd1e9125a1af606abeae22b45d7770e0feea91600ba1ff0ddab50d size: 1792


With our docker image in a registry, it is available to our cluster, so we can deploy!

In [24]:
!kubectl apply -f iris_classifier/sklearn_iris_deployment.yaml

seldondeployment.machinelearning.seldon.io/seldon-deployment-example created


You can check the status of your deployment. 

In [25]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=seldon-deployment-example \
                                 -o jsonpath='{.items[0].metadata.name}')

Waiting for deployment "seldon-92a927e5e90d7602e08ba9b9304f70e8" rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-92a927e5e90d7602e08ba9b9304f70e8" successfully rolled out


Once the deployment is ready, you will need to port-forward the pod to your localhost in order check the request. That can be done wiht kubectl port-forward command 
```bash 
kubectl port-forward $(kubectl get pods -l seldon-app=seldon-deployment-example-sklearn-iris-predictor -o jsonpath='{.items[0].metadata.na}') 9000:9000`
```

You must run this command in a separate window because it will need to run while we curl the endpoint. 

In [49]:
!curl -s http://localhost:9000/predict -H "Content-Type: application/json" -d '{"data":{"ndarray":[[5.964,4.006,2.081,1.031]]}}'
        
        

{"data":{"names":["t:0","t:1","t:2"],"ndarray":[[0.9548873249364169,0.04505474761561406,5.792744796895234e-05]]},"meta":{}}


You have successfully created a seldon endpoint on kubernetes! 

In [53]:
## Cleanup
!kubectl delete -f sklearn_iris_deployment.yaml


seldondeployment.machinelearning.seldon.io "seldon-deployment-example" deleted


### Conclusion 
In this quick example, we scratched the surface of seldon-core by deploying a simple model endpoint on kubernetes. 
